In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


In [2]:
# Generate tain and test data
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=8, 
                    n_redundant=0, n_clusters_per_class=2)
Y = np.array([Y, -(Y-1)]).T  # The model currently needs one column for each class
X, X_test, Y, Y_test = train_test_split(X, Y)

In [3]:

# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 10 # 2nd layer number of features
n_input = 10 # Number of feature
n_classes = 2 # Number of classes to predict

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [4]:

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
#cross_entropy = tf.reduce_sum(- y * tf.log(pred) - (1 - y) * tf.log(1 - pred), 1)
#cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [5]:
"""
Here is the TF Saver object to store the model
"""
saver = tf.train.Saver()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # batch_y.shape = (batch_y.shape[0], 1)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)
    print "Optimization Finished!"





Epoch: 0001 cost= 2.991616639
Epoch: 0002 cost= 0.751835973
Epoch: 0003 cost= 0.481648449
Epoch: 0004 cost= 0.386603158
Epoch: 0005 cost= 0.333575615
Epoch: 0006 cost= 0.298181544
Epoch: 0007 cost= 0.272505879
Epoch: 0008 cost= 0.253354802
Epoch: 0009 cost= 0.238239394
Epoch: 0010 cost= 0.225616478
Model saved in file: /tmp/model.ckpt
Optimization Finished!


In [6]:
# Launch the graph
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    print("Now calculating all the scores and metrics and doing prediction")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: X_test, y: Y_test})
    global result 
    result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})
    
    
    
    probab = pred
    
    
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred, probab = sess.run([accuracy, y_p, probab], feed_dict={x:X_test, y:Y_test})
    
    probab_out = probab.tolist()
    
    print(probab_out[:5])
    print (y_pred[:5])
    
    Y_test = [not i for i in Y_test[:,0]]    
    y_pred =  y_pred.tolist()
    
    
    
    

    
    (test_precision,test_recall,test_fscore,test_support)=precision_recall_fscore_support(Y_test, y_pred, beta=1.0, labels=None,
	                            pos_label=1, average=None,
	                            warn_for=('precision', 'recall',
	                                      'f-score'),
	                            sample_weight=None)

    Test = (test_precision,test_recall,test_fscore,test_support)
    print "TEST RESULT:"
    print"=============="
    print "\n"
    print "PRECISION:"
    print(Test[0])
    print "\n"
    print "RECALL:"
    print(Test[1])
    print "\n"
    print "F1-score:"
    print(Test[2])
    print "\n"	
    print "SUPPORT:"
    print(Test[3])
    

Model restored.
Now calculating all the scores and metrics and doing prediction
Accuracy: 0.92176
[[-6.518826484680176, -9.938179016113281], [-3.1502633094787598, -9.780601501464844], [-0.7854443788528442, 1.0382291078567505], [10.784098625183105, 2.4796228408813477], [-0.835651159286499, 1.0489269495010376]]
[0 0 1 0 1]
TEST RESULT:


PRECISION:
[ 0.9228659  0.9206727]


RECALL:
[ 0.91960122  0.92389747]


F1-score:
[ 0.92123067  0.92228226]


SUPPORT:
[6219 6281]
